In [278]:
from reader import ebook
from main import read_list,read_json
from typing import Optional
import requests
from dotenv import load_dotenv
import json
import os


In [255]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"


In [256]:
role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and Ready to parse. nothing before or after the json file)
Input:  
1.Current Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
4.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize and explain the chapter in detail in {Length} words or less, integrating context and key developments from previous chapters and create a self containing summary and explaination. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary and explination of the current chapter in "Length" words context of previous chapters. Use previus chapter summary as context",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}

example:
{
  "summary": "The chapter opens with the protagonist, Monsieur Meursault, receiving a telegram about his mother's death. 
  Despite the news, he reacts with emotional detachment, pondering the ambiguity of the exact day of her passing.
  He prepares to attend the funeral, arranging leave from work, but notes his employer's subtle annoyance.
  His journey to the Home for Aged Persons in Marengo reveals the intense heat of the Algerian climate and his interactions with others, 
  such as a soldier on the bus and the doorkeeper. 
  At the Home, Meursault encounters the warden, who seems both judgmental and understanding about the circumstances leading to his mother's stay. 
  The chapter concludes with Meursault entering the mortuary, where his lack of emotion continues as he declines to view the body, further emphasizing his detachment.",
  
  "characters": {
    "Monsieur Meursault": "A young man, emotionally detached and indifferent. He appears practical, with a reserved demeanor. His body language reflects discomfort when dealing with social interactions, such as prolonged handshakes or unnecessary apologies.",
    "The Doorkeeper": "A man in his sixties, hale and hearty with a cheerful demeanor. He has ruddy cheeks, blue eyes, and a white mustache. Though officially a doorkeeper, he sees himself as an authority figure among the residents.",
    "The Warden": "A small, older man with gray hair and watery blue eyes. He wears a Legion of Honor rosette and has a formal yet slightly judgmental attitude. He holds a prolonged handshake, perhaps signaling awkwardness or an attempt at connection."
  },
  "places": {
    "The Home for Aged Persons": "Located in Marengo, fifty miles from Algiers, the Home has a formal and slightly oppressive atmosphere. The environment is marked by a clean mortuary, whitewashed walls, and a courtyard where elderly residents gather. The weather is intensely hot, reflecting the harsh Algerian climate.",
    "Marengo": "A small village with a rural, subdued atmosphere. The intense afternoon heat creates a sense of lethargy and discomfort. The Home itself is slightly isolated, requiring a mile-long walk from the bus stop."
  }
}
'''

In [257]:
def msg(text: str, context: str, length:int, characters: dict = {}, places: dict = {}) -> list:


    message = [
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": json.dumps({
                "Lenght":length,
                "past_context": context,
                "Current_Chapter": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

def get_summ(messages) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None
    
def count_words(text:str)->int:
    percent=20
    return (len(text.replace("\n","").split(" "))*percent//20)

def count_imgs(text: str)->int:
    return round(len(text.replace("\n","").split(" "))/300)

In [258]:
book=ebook("./books/HP.epub")
chapters = book.get_chapters()
_,text=zip(*chapters[:5]) 



In [259]:
print(len(
    """The chapter opens with the author reflecting on his childhood, sharing a story about a drawing of a boa constrictor digesting an elephant. This anecdote serves as a metaphor for the author's feelings towards adults, whom he feels do not understand children and often require excessive explanations. The author then shares his experiences living among adults, expressing his disappointment with their lack of lucidity. He talks about a strange encounter in the Sahara desert six years ago, where he was stranded and nearly died. At dawn, he was awakened by a mysterious, child-like voice that asked him to draw a sheep. The chapter ends with the author describing his surprise and confusion at this encounter."""
    ))

708


In [260]:

output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }

In [262]:
for idx,i in enumerate(text):
    
    context=output_dict[idx]["summary"]
    length=count_words(i)
    characters=output_dict[idx]["characters"]
    places=output_dict[idx]["places"]
    
    mes=msg(i,context,length,characters,places)
    
    summary_characters=get_summ(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")
    

chapter: 0 done
chapter: 1 done
chapter: 2 done
chapter: 3 done
chapter: 4 done


In [277]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Harry Potter: An old-fashioned hero, yet appealing to both children and adults. His character is defined by his ability to learn from choices, rather than relying solely on abilities.

Places:
The World of Harry Potter: A fictional world where the story of Harry Potter unfolds. It is characterized by magic, adventure, and a setting that appeals to both children and adults.

Summary:
In this chapter, the story introduces the world of Harry Potter, a bestselling children's novel with an international appeal. The narrative sets the scenario of an old-fashioned hero named Harry Potter, who learns that choices reveal one's true character more than abilities. The chapter emphasizes the importance of magic and adventure in the story.

-------------------------------------------------------------------------------------------------------------------
CHAPTER 2 :

Characters:
Harry Potter: A young hero living with the Dursleys. He is special, having survived an attack fr

In [265]:
for key, val in output_dict.items():
    print(val["summary"])

This is the first chapter
In this chapter, the story introduces the world of Harry Potter, a bestselling children's novel with an international appeal. The narrative sets the scenario of an old-fashioned hero named Harry Potter, who learns that choices reveal one's true character more than abilities. The chapter emphasizes the importance of magic and adventure in the story.
In this chapter, the story introduces Harry Potter, a young hero living with his mean and mundane Muggle aunt and uncle, the Dursleys. The narrative emphasizes the stark contrast between the magical world of Harry Potter and the mundane world of the Dursleys. The chapter highlights the arrival of Harry's guardian, Albus Dumbledore, who brings Harry a letter and a mysterious baby boy, whom Dumbledore leaves with the Dursleys. The chapter concludes with Dumbledore and his companion, Professor McGonagall, discussing the events surrounding Harry's arrival and the recent disappearance of Voldemort.
In this chapter, Harry

## IMAGE GEN

In [266]:
#imports
import time

In [267]:
from main import read_list
from huggingface_hub import InferenceClient

##stabilityai
client = InferenceClient(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    token=API,
)


def scene_msg(text: str,num) -> list:
    message = [
        {
            "role": "system",
            "content": ''' IMPORTANT-> ONLY OUTPUT IN PYTHON-LIST.
                You are a text-to-image prompt generator. 
                Your task is to analyze the provided input text and identify distinct scenes within it.
                For each scene, create a detailed prompt suitable for generating an image.
                Call characters by their respacted names.
                Ensure that the number of prompts does not exceed the specified limit provided as the second input.

                ### Format:
                1. **Input:**
                   - `text`: A block of narrative text containing multiple scenes.
                   - `num`: An integer representing the maximum number of scenes to generate prompts for.

                2. **Output:**
                   - A Python list containing the generated prompts. The list should be formatted as follows:
                     ```
                     ["prompt", "prompt", ..., "prompt"]
                     ```

                ### Instructions:
                - Identify key changes in location, characters, or significant actions to define separate scenes.
                - Use descriptive language to paint a vivid picture of each scene in the prompt.
                - Limit the number of prompts to the value of `num`.
                
                   ''',
        },
        
        {
            "role": "user",
            "content": f"""Num={num}, TEXT: {text}""",  # This should be your input text that describes the scenes
        },]
        
    return message



def get_scene(text: str,num_of_img:int) -> Optional[str]:
    messages = scene_msg(text,num_of_img)
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
    }
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None
    
def basic_llm_req(text: str) -> Optional[str]:
    messages = [
        {
            "role":"system",
            "content":"DO As Asked in The Input"
        },
        {
            "role":"user",
            "content":text
        }
    ]
    
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
    }
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None
    
# @lru_cache(maxsize=1024)
def get_images(key, text,characters,places,style):
    image = client.text_to_image(
        f"Generate a illustration for a book given the prompt,use provided character place descriptions if needed. Prompt: {text}, characters: {characters}, places:{places}// style:{style} ",
        height=528,
        width=720,
    )
    image.save(f"./output/LP_{key}.png")




In [268]:
outputs=[]
for i in text:
    num_of_image=count_imgs(i)
    inputs=i.replace("\n"," ")
    scenes=get_scene(inputs,num_of_image)
    outputs.append(read_list(scenes))



In [269]:
output_string=""
for key,val in output_dict.items():
    val_content=val["summary"].replace("\n"," ")
    if key==0:
        continue
    output_string= f'''{output_string}... Chapter{key}: {val_content}'''


In [270]:
style_prompt='''Prompt:

"Analyze the following story and provide a list of image style tags that would best suit its themes, settings, and overall mood. 
The response should include the style, period, type of art, color palette etc. 4 tags per entry.dont explain it just give tags.

Response Format:

Style: Realism, Impressionism, Surrealism,etc.
Period:Romantic, Modern, Contemporary,etc.
Type:Landscape, Portrait, Abstract,etc.
Color Palette:Warm tones, Cool colors, Monochromatic,etc
Mood:Serene, Dramatic, Melancholic,etc.

Story
'''

style=basic_llm_req(f'''{style_prompt}:{output_string} ''')

In [275]:
print(style)

Style: Realism, Fantasy, Surrealism, Magical Realism
Period: Modern, Contemporary
Type: Portrait (Harry Potter), Landscape (Zoo), Abstract (Magical Occurrence)
Color Palette: Warm tones (Harry Potter, Hagrid), Cool colors (Dursleys' house), Vibrant (Magical World)
Mood: Melancholic (Dursleys' household), Anticipation (Harry's future), Dramatic (Hagrid's arrival)


In [ ]:
output_dict

In [273]:
for idx,i in enumerate(outputs):
    chars=output_dict[idx+1]["characters"]
    places=output_dict[idx+1]["places"]
    for jdx,j in enumerate(i):
        get_images(key=f"chapter{idx}_scene{jdx+1}",text=j,characters=chars,places=places,style=style)
        print(f"chapter{idx}_scene{jdx+1} saved")
        time.sleep(30)
        
    

chapter0_scene1 saved
chapter0_scene2 saved
chapter0_scene3 saved
chapter0_scene4 saved
chapter1_scene1 saved
chapter1_scene2 saved
chapter1_scene3 saved
chapter1_scene4 saved
chapter1_scene5 saved
chapter1_scene6 saved
chapter1_scene7 saved
chapter1_scene8 saved
chapter1_scene9 saved
chapter1_scene10 saved
chapter2_scene1 saved
chapter2_scene2 saved
chapter2_scene3 saved
chapter2_scene4 saved
chapter2_scene5 saved
chapter2_scene6 saved
chapter2_scene7 saved
chapter2_scene8 saved
chapter2_scene9 saved
chapter2_scene10 saved
chapter2_scene11 saved
chapter2_scene12 saved
chapter2_scene13 saved
chapter2_scene14 saved
chapter2_scene15 saved
chapter2_scene16 saved
chapter2_scene17 saved
chapter2_scene18 saved
chapter2_scene19 saved
chapter3_scene1 saved
chapter3_scene2 saved
chapter3_scene3 saved
chapter3_scene4 saved
chapter3_scene5 saved
chapter3_scene6 saved
chapter3_scene7 saved
chapter3_scene8 saved
chapter3_scene9 saved
chapter3_scene10 saved
chapter3_scene11 saved
chapter3_scene12 sa

KeyboardInterrupt: 